## first model

This is the first try (after the proof of concept) to create a model which is capable of detecting all three classes correctly.

In [1]:
import sys
from os.path import join

sys.path.append('..')

raw = join('..', 'data', 'raw')
interim = join('..', 'data', 'interim')
processed = join('..', 'data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [300,100,100])

['..\\data\\processed\\train\\n',
 '..\\data\\processed\\validation\\n',
 '..\\data\\processed\\test\\n',
 '..\\data\\processed\\train\\o',
 '..\\data\\processed\\validation\\o',
 '..\\data\\processed\\test\\o',
 '..\\data\\processed\\train\\x',
 '..\\data\\processed\\validation\\x',
 '..\\data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = join(processed, 'train')
validation_dir = join(processed, 'validation')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary',
    save_to_dir=interim)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(32, 32),
    batch_size=20,
    class_mode='binary')

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Dense(64, 'relu', input_shape=(32, 32, 3)))
model.add(layers.Dense(32, 'relu'))
model.add(layers.Dense(16,'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32, 32, 64)        256       
_________________________________________________________________
dense_1 (Dense)              (None, 32, 32, 32)        2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32, 32, 16)        528       
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 49155     
Total params: 52,019
Trainable params: 52,019
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

log_dir = join('..', 'logs')

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=callbacks)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100/100 [==============================] - 14s 137ms/step - loss: 1.0958 - acc: 0.3835 - val_loss: 1.0877 - val_acc: 0.3580
Epoch 2/10
100/100 [==============================] - 12s 123ms/step - loss: 1.0564 - acc: 0.4925 - val_loss: 1.0430 - val_acc: 0.5300
Epoch 3/10
100/100 [==============================] - 12s 125ms/step - loss: 0.9778 - acc: 0.6050 - val_loss: 0.9451 - val_acc: 0.6080
Epoch 4/10
100/100 [==============================] - 12s 121ms/step - loss: 0.8457 - acc: 0.6740 - val_loss: 0.8245 - val_acc: 0.6690
Epoch 5/10
100/100 [==============================] - 13s 129ms/step - loss: 0.6675 - acc: 0.7745 - val_loss: 0.7039 - val_acc: 0.7030
Epoch 6/10
100/100 [==============================] - 13s 130ms/step - loss: 0.5176 - acc: 0.8330 - val_loss: 0.6797 - val_acc: 0.7080
Epoch 7/10
100/100 [==============================] - 12s 122ms/step - loss: 0.4302 - acc: 0.856

In [6]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 5052), started 1 day, 6:26:33 ago. (Use '!kill 5052' to kill it.)